In [19]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [53]:
"""
요약
ConversationKGMemory 는
대화 중 엔티티의 Knowledge graph를 만든다

중요한 것을 뽑는 요약본
"""
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

In [58]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

In [59]:
def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

In [60]:
add_message("Hi I'm dooseop, I live in Seoul", "Wow that is so cool!!")

In [61]:
memory.load_memory_variables({"input":"What is Seoul"})

{'history': []}

In [62]:
add_message("Dooseop likes kimchi", "Wow that is so cool!!")

In [63]:
memory.load_memory_variables({"input":"what does Dooseop like"})

{'history': [SystemMessage(content='On Dooseop: Dooseop likes kimchi.')]}